Import all needed resources

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from Evaluations import evaluate_classification_model
from sklearn.linear_model import LogisticRegression
from Evaluations import evaluate_classification_model

from Prep_Data import load_student_data, explore_student_data, prepare_data_for_training, engineer_features
from Analysis import univariate_analysis, bivariate_analysis


ModuleNotFoundError: No module named 'Evaluations'

Loading Data, cleaning data, removing outliers, removing redundant data, feature engineering and splitting test and training data

In [ ]:

filepath = "../Data/Student_performance_data.csv"
df = load_student_data(filepath)
df = engineer_features(df)
X_train, X_test, y_train, y_test = prepare_data_for_training(df)


Data Understanding

In [ ]:
explore_student_data(df)

Exploratory Data Analysis

In [ ]:
univariate_analysis(df)
bivariate_analysis(df)

Evaluation Metrics and model trainng and model usage

In [ ]:

print("\nLogistic Regression:")
lr = LogisticRegression(class_weight='balanced', random_state=42, max_iter=10000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
# Evaluate model
evaluate_classification_model(y_test, lr_pred)

# Train and test Random Forest
print("\nRandom Forest:")
rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
# Evaluate model
evaluate_classification_model(y_test, rf_pred)


# Train and test XGBoost
print("\nXGBoost:")
# Adjust for class imbalance
scale_weight = len(y_train[y_train == 4]) / len(y_train[y_train != 4])
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

evaluate_classification_model(y_test, xgb_pred)

In [ ]:

import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print("\nNeural Network:")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode target
y_train_onehot = to_categorical(y_train, num_classes=5)
y_test_onehot = to_categorical(y_test, num_classes=5)

# Compute class weights for imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Define neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train_scaled, y_train_onehot, epochs=20, batch_size=32,
                   validation_split=0.2, class_weight=class_weight_dict, verbose=1)

# Predict on test set
y_pred_onehot = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_onehot, axis=1)
print("Neural Network predictions:", y_pred[:10])

# Evaluate model
evaluate_classification_model(y_test, y_pred)

# Plot loss curves
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Neural Network Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()